In [1]:
import numpy as np, matplotlib.pyplot as plt, pandas as pd
pd.set_option('display.max_rows', 40)

from vivarium_research_prl.noise import corruption, fake_names
from vivarium_research_prl.find_kids import datasets

!date
!whoami
!uname -a
!pwd

Thu 08 Dec 2022 04:15:11 PM PST
ndbs
Linux int-slurm-sarchive-p0009 5.4.0-88-generic #99-Ubuntu SMP Thu Sep 23 17:29:00 UTC 2021 x86_64 x86_64 x86_64 GNU/Linux
/mnt/share/code/ndbs/vivarium_research_prl/linkage/wic_case_study


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# from vivarium_research_prl import noise
# import datetime as dt

# Load census data and state table for year 2020

In [4]:
project_output_dir = '/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop'
output_subdir = 'results/special_last_names/florida/2022_10_14_10_49_32/population_table/'
output_dir = f'{project_output_dir}/{output_subdir}'

!ls -l $output_dir

total 32224
-rw-rw-r-- 1 albrja   IHME-Simulationscience 12622072 Oct 20 23:08 decennial_census.hdf
-rwxrwxrwx 1 beatrixh IHME-Simulationscience 20364830 Nov 14 16:42 state_table.hdf


In [5]:
decennial_census_path = f'{output_dir}/decennial_census.hdf'
state_table_path = f'{output_dir}/state_table.hdf'

df_census_orig = pd.read_hdf(decennial_census_path, 'year_2020')
df_state_table = pd.read_hdf(state_table_path, 'ymd_2020_4_1')
print(f'{df_census_orig.shape=}')
print(f'{df_state_table.shape=}')

df_census_orig.shape=(47444, 10)
df_state_table.shape=(50000, 27)


In [11]:
47444/50000

0.94888

# Get a random seed and Generator

In [6]:
sq = np.random.SeedSequence()
sq.entropy

334555326833832947908816060887211517881

In [7]:
sq = np.random.SeedSequence(66624024798819663709061712465147975287)
print(sq.entropy)
rng = np.random.default_rng(sq)
rng

66624024798819663709061712465147975287


Generator(PCG64) at 0x7F34B0FECE40

# Generate WIC data and decennial census data

In [10]:
df_census = datasets.generate_census_data(df_state_table, 0.95, 0.90, rng)
df_wic = datasets.generate_wic_data(df_state_table, rng)
print(f"{df_census.shape=}")
print(f"{df_wic.shape=}")

df_census.shape=(47550, 10)
df_wic.shape=(637, 10)


In [12]:
47550/50000

0.951

In [13]:
df_census

,first_name,middle_initial,last_name,date_of_birth,age,sex,race_ethnicity,relation_to_household_head,address,zipcode
0,Margaret,J,Clark,1951-07-27,68.0,Female,Black,Reference person,"1344 winoka rd brooksville, fl",34601
1,Jeffrey,V,Littlejohn,1967-05-03,52.0,Male,Black,Reference person,"927 23rd st clearwater, fl",34698
2,Briana,A,Jackson,2006-09-07,13.0,Female,Black,Biological child,"927 23rd st clearwater, fl",34698
3,Benjamin,D,Cox,1998-10-21,21.0,Male,Black,Stepchild,"927 23rd st clearwater, fl",34698
4,Willie,J,Tucker,1947-10-09,72.0,Male,White,Reference person,"8904 167th place fleming island, fl",32003
...,...,...,...,...,...,...,...,...,...,...
49995,Brian,N,Roberts,1984-07-05,35.0,Male,White,Institutionalized GQ pop,"114 s frnt st fort myers, fl",33919
49996,Nathaniel,J,Campbell,1941-01-08,79.0,Male,White,Institutionalized GQ pop,"2210 henn hyde rd ne hollywood, fl",33021
49997,Christian,C,Rosales,1983-12-16,36.0,Male,Latino,Institutionalized GQ pop,"701 haber rd vero beach, fl",32968
49998,Phillip,J,Morton,1985-06-11,34.0,Male,White,Institutionalized GQ pop,"114 s frnt st fort myers, fl",33919


In [14]:
df_census.dtypes

first_name                     object
middle_initial                 object
last_name                      object
date_of_birth                  object
age                           float64
sex                            object
race_ethnicity                 object
relation_to_household_head     object
address                        object
zipcode                        object
dtype: object

In [15]:
df_wic

,first_name,middle_name,last_name,date_of_birth,sex,race_ethnicity,address,zipcode,household_id,wic_id
166,Everleigh,Ireland,Dietzman,2020-02-25,Female,White,"1286 county road 185 pembroke pines, fl",33024,85,1
323,Gunner,Liam,Parkinson,2020-03-03,Male,White,"600 n maranantha rd hialeah, fl",33016,157,2
492,Keyla,Heavenly,Musse,2019-03-02,Female,Black,10079 fair lane aptmnt no 113l indian rocks be...,33785,233,3
531,Sophia,Jayla,Grimm,2018-09-29,Female,White,"519 jewell drive aptmnt 10068 rockdale, fl",33175,249,4
792,Khloe,Liliana,Jackson,2020-01-27,Female,White,"974 kini court aptmnt 366 vero beach, fl",32962,364,5
...,...,...,...,...,...,...,...,...,...,...
48298,Joshua,Zion,Cortez,2019-01-18,Male,Latino,"45 glen devin drive stuart, fl",34997,20397,633
48425,Luke,Hunter,Gonzalez,2018-04-07,Male,Latino,"4877 pointillist ct chiefland, fl",32626,20448,634
48437,Damani,Hunter,Zard,2018-09-12,Male,White,"255 e a st largo, fl",33772,20451,635
48456,Liam,Emmett,Sardone,2017-01-08,Male,White,"107 brown ave st. petersburg, fl",33704,20458,636


In [16]:
df_wic.dtypes

first_name        object
middle_name       object
last_name         object
date_of_birth     object
sex               object
race_ethnicity    object
address           object
zipcode           object
household_id       int64
wic_id             int64
dtype: object

# See how many kids are in census data

Only 2236. If I restrict to these, it would be a very small linking task. However, if ages and birthdates are noisy, then we'd have to search the rest of the records after matching these.

    
## Questions about linking

1. If ages/birthdays are noisy, what's a good strategy for finding missing kids in a "cascade" approach?

In [17]:
(df_census.age < 5).sum()

2242

# Add noise to data

## Noise for census in priority order:

1. First name (keyboard, phonetic, fake)
1. Last name (keyboard, phonitic, fake)
1. Date of birth (switch month/day, wrong number or person)
1. Zipcode
1. Address
1. Sex
1. Age
1. Race/Ethnicity
1. Middle initial

## Noise for WIC data in priority order:

1. First name (keyboard, phonetic, fake, OCR)
1. Last name (keyboard, phonitic, fake, OCR)
1. Date of birth (switch month/day, wrong number or person, OCR)
1. Zipcode
1. Address
1. Sex
1. Race/Ethnicity
1. Middle name
1. Household ID and WIC ID??

## Questions about adding noise:

1. What are good default values for the token corruption probabilities?

1. How should I handle combining the row corruption probability with the token corruption probabilities? To get a desired fraction of rows corrupted, you need to modify a larger fraction of rows, e.g. divide the row fraction by the token corruption probability or something. (The question is not about how to do the math, but about where to do the math, or if we need to do the math.)

1. How should I handle the datetime64 datatype in date of birth?

    1. Does Splink (and other packages) handle this datatype directly, or do they need to be converted to strings?
    1. How do I switch the month and day in a fraction of rows?
    1. If a swapped month and day lead to an invalid date in the format YYYY-MM-DD, do we keep the invalid date, or should this get fixed automatically since it's obviously invalid? Would correcting obvious errors like this be part of the preprocessing pipeline? E.g., just like there's name and address standardization, there would be date standardization.
    1. Maybe the problem is that the data is already "too clean" because I'm just starting with the ground truth, which is stored as datetime64 objects in a standard format (e.g., so I can't switch the month and day to invalid values without getting an error or changing the format of every row). Perhaps part of the initial "data creation" process should be to convert these to strings, which have less structure and may be easier to manipulate, and then one of the first preprocessing steps would be to standardize the dates, eliminiating any impossible values.

1. What **other** ways could birthdays be corrupted besides swapping month and day?

1. There are **lots** of possible parameters to specify for noise functions. Where is the best place to specify these, how do I pass them around to different parts of the code, and how do I make a user-friendly (for now, me-friendly) interface to handle them?

1. How should I add noise to zipcodes?

1. How should I add noise to addresses?

    1. Also, should I try to do any address standardization? Are there available packages that already do this?
    1. Can I replace some addresses with (possibly different) addresses from the 2030 census file to make matching harder?

## Write function to swap month and day for date of birth

In [18]:
df_census.date_of_birth

0        1951-07-27
1        1967-05-03
2        2006-09-07
3        1998-10-21
4        1947-10-09
            ...    
49995    1984-07-05
49996    1941-01-08
49997    1983-12-16
49998    1985-06-11
49999    1928-08-14
Name: date_of_birth, Length: 47550, dtype: object

In [22]:
df_census.date_of_birth[0]

'1951-07-27'

In [27]:
pattern = "(?P<year>\d+).*"
df_census.date_of_birth.str.extract(pattern)

,year
0,1951
1,1967
2,2006
3,1998
4,1947
...,...
49995,1984
49996,1941
49997,1983
49998,1985


In [61]:
def swap_month_day(date_col, date_format="yyyy-mm-dd"):
    date_format = date_format.lower()
    if date_format=="yyyy-mm-dd":
        pattern = "^(?P<year>\d{4})-(?P<month>\d{2})-(?P<day>\d{2})$"
    else:
        raise ValueError(f"unsupported date format: {date_format}")
    date_df = date_col.str.extract(pattern)
    if date_format=="yyyy-mm-dd": 
        new_date_col = (
            date_df['year'] + '-' + date_df['day'] + '-' + date_df['month']
        )
    return new_date_col

swap_month_day(df_census.date_of_birth)

0        1951-27-07
1        1967-03-05
2        2006-07-09
3        1998-21-10
4        1947-09-10
            ...    
49995    1984-05-07
49996    1941-08-01
49997    1983-16-12
49998    1985-11-06
49999    1928-14-08
Length: 47550, dtype: object

# Write function to add noise to zipcode

In [265]:
def miswrite_zipcode(zipcode, first3_prob=0.001, last2_prob=0.02, random_state=None):
    rng = np.random.default_rng(random_state)
    threshold = np.array(3*[first3_prob] + 2*[last2_prob])
    replace = rng.random(5) < threshold
    random_digits = rng.choice(10, 5)
    digits = np.where(replace, random_digits, list(zipcode))
    return ''.join(digits)

def miswrite_zipcode_vec(zipcode, first3_prob=0.001, last2_prob=0.02, random_state=None):
    rng = np.random.default_rng(random_state)
    threshold = np.array(3*[first3_prob] + 2*[last2_prob])
    shape = (len(zipcode), 5)
    replace = rng.random(shape) < threshold
    random_digits = rng.choice(10, shape)
    zipcode_array = np.stack(zipcode.str.split(''))[:,1:-1]
    digits_array = np.where(replace, random_digits, zipcode_array)
    digits = pd.Series(list(digits_array), index=zipcode.index, name=zipcode.name)
    return digits.str.join('')

miswrite_zipcode('85737', .1, .3, rng)

'85736'

### Test some code for `miswrite_zipcode` function

In [106]:
rng.choice(10, 20)

array([9, 2, 5, 6, 0, 4, 3, 7, 4, 6, 4, 3, 8, 8, 7, 6, 4, 5, 8, 7])

In [103]:
list('39393')

['3', '9', '3', '9', '3']

In [94]:
threshold = np.array(3*[.3] + 2*[.02])
unif = rng.random((7,5))
print(unif)
unif < threshold

[[0.80999752 0.13840215 0.58733314 0.56220657 0.24811742]
 [0.56571948 0.47190002 0.55887749 0.39123623 0.28567498]
 [0.5748391  0.65717589 0.83813844 0.54110045 0.71419805]
 [0.05892595 0.32170491 0.82395045 0.89363152 0.20042659]
 [0.57337898 0.23680694 0.16549837 0.66094035 0.36552605]
 [0.80944619 0.09658531 0.10965866 0.48530541 0.52994211]
 [0.45464244 0.06691927 0.06899682 0.36799075 0.18271859]]


array([[False,  True, False, False, False],
       [False, False, False, False, False],
       [False, False, False, False, False],
       [ True, False, False, False, False],
       [False,  True,  True, False, False],
       [False,  True,  True, False, False],
       [False,  True,  True, False, False]])

### Figure out vectorized way to split zipcode into digits, replace some, then recombine

In [131]:
zips = pd.Series(['11234', '89930', '55783'])
zips.str.split('').str.join('')

0    11234
1    89930
2    55783
dtype: object

In [139]:
np.asarray(zips.str.split(''))

array([list(['', '1', '1', '2', '3', '4', '']),
       list(['', '8', '9', '9', '3', '0', '']),
       list(['', '5', '5', '7', '8', '3', ''])], dtype=object)

In [159]:
# https://stackoverflow.com/questions/40824601/how-to-convert-a-series-of-arrays-into-a-single-matrix-in-pandas-numpy
a = np.stack(zips.str.split('').to_numpy())
a


array([['', '1', '1', '2', '3', '4', ''],
       ['', '8', '9', '9', '3', '0', ''],
       ['', '5', '5', '7', '8', '3', '']], dtype='<U1')

In [160]:
a[:,1:-1] # strip off the empty strings at the ends

array([['1', '1', '2', '3', '4'],
       ['8', '9', '9', '3', '0'],
       ['5', '5', '7', '8', '3']], dtype='<U1')

In [178]:
pd.Series(list(a))

0    [, 1, 1, 2, 3, 4, ]
1    [, 8, 9, 9, 3, 0, ]
2    [, 5, 5, 7, 8, 3, ]
dtype: object

### See how long it takes to convert a Series of lists to a 2D NumPy array

In [173]:
np.stack(df_state_table.loc[five_digits, 'zipcode'].str.split('').to_numpy())

array([['', '3', '4', ..., '0', '1', ''],
       ['', '3', '4', ..., '9', '8', ''],
       ['', '3', '4', ..., '9', '8', ''],
       ...,
       ['', '3', '2', ..., '6', '8', ''],
       ['', '3', '3', ..., '1', '9', ''],
       ['', '3', '2', ..., '6', '8', '']], dtype='<U1')

In [172]:
np.stack(df_state_table.loc[five_digits, 'zipcode'].str.split(''))

array([['', '3', '4', ..., '0', '1', ''],
       ['', '3', '4', ..., '9', '8', ''],
       ['', '3', '4', ..., '9', '8', ''],
       ...,
       ['', '3', '2', ..., '6', '8', ''],
       ['', '3', '3', ..., '1', '9', ''],
       ['', '3', '2', ..., '6', '8', '']], dtype='<U1')

In [155]:
# Must filter to 5-digit zipcodes for this to work -- see below
five_digits = df_state_table['zipcode'].str.len() == 5
%timeit np.stack(df_state_table.loc[five_digits, 'zipcode'].str.split('').array)

197 ms ± 2.57 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [156]:
%timeit np.stack(df_state_table.loc[five_digits, 'zipcode'].str.split('').to_numpy())

127 ms ± 675 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [174]:
%timeit np.stack(df_state_table.loc[five_digits, 'zipcode'].str.split(''))

125 ms ± 887 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### See why some zipcodes are not 5 digits

I got an "arrays must have same length" error from `numpy.stack` when I first tried to run the time tests above, before I filtered down to 5 digits.

Looks like there are three nonstandard formats that show up in a very small fraction of rows...

After [checking with Zeb](https://ihme.slack.com/archives/C02KUQ9LX32/p1670632967272579) about why (apparently this is what the real data looks like), I think the best thing is to omit these from the state table when loading it -- eventually we plan to have standardized 5-digit zipcodes before noise is added.

In [157]:
df_state_table['zipcode'].str.len().value_counts()

5     49979
7        11
8         8
10        2
Name: zipcode, dtype: int64

In [158]:
df_state_table.loc[df_state_table['zipcode'].str.len() != 5]

,tracked,middle_name,sex,relation_to_household_head,race_ethnicity,housing_type,exit_time,last_name,state,ssn,...,years_of_life_lost,cause_of_death,zipcode,address,parent_id,last_birth_time,employer_id,employer_name,employer_zipcode,employer_address
12102,True,Bonnie,Female,Reference person,White,Standard,NaT,Porter,12,294-19-7040,...,0.0,not_dead,cp 33440,"2501 bdwy st clewiston, fl",-1,2019-03-04 18:00:00,84,not implemented,32164,"86 timberbrook cir palm coast, fl"
14637,True,Kieara,Female,Reference person,White,Standard,NaT,Sessions,12,240-48-3352,...,0.0,not_dead,cp34110,"653 middle cr rd naples, fl",-1,2019-03-04 18:00:00,-1,unemployed,NA,NA
16971,True,Connie,Female,Reference person,White,Standard,NaT,Matuszewski,12,766-95-3849,...,0.0,not_dead,cp33415,"1308 blue vale way unincorporated, fl",-1,2019-03-04 18:00:00,-1,unemployed,NA,NA
16972,True,Harrison,Male,Opp-sex spouse,White,Standard,NaT,Barnum,12,872-50-3326,...,0.0,not_dead,cp33415,"1308 blue vale way unincorporated, fl",-1,NaT,45,not implemented,34744,"10780 rivercrest dr kissimmee, fl"
16973,True,Jacob,Male,Biological child,White,Standard,NaT,Minas,12,756-86-1416,...,0.0,not_dead,cp33415,"1308 blue vale way unincorporated, fl",-1,NaT,-1,unemployed,NA,NA
16974,True,Javier,Male,Biological child,White,Standard,NaT,Hinchliff,12,716-26-5094,...,0.0,not_dead,cp33415,"1308 blue vale way unincorporated, fl",-1,NaT,17,not implemented,33440,"412 58 stree harlem, fl"
23973,True,Kelli,Female,Reference person,White,Standard,NaT,Olson,12,144-66-9721,...,0.0,not_dead,32223-1647,"525 canary dr jacksonville, fl",-1,2019-03-04 18:00:00,-1,unemployed,NA,NA
23974,True,Brian,Male,Other nonrelative,White,Standard,NaT,Jefferis,12,405-86-9101,...,0.0,not_dead,32223-1647,"525 canary dr jacksonville, fl",-1,NaT,-1,unemployed,NA,NA
27551,True,Robbie,Male,Reference person,White,Standard,NaT,Holubar,12,462-32-6995,...,0.0,not_dead,cp 33433,"6819 honeysuckle trl unincorporated, fl",-1,NaT,-1,unemployed,NA,NA
27552,True,Andrea,Female,Opp-sex spouse,Latino,Standard,NaT,Abernathy,12,042-99-2615,...,0.0,not_dead,cp 33433,"6819 honeysuckle trl unincorporated, fl",-1,2019-03-04 18:00:00,80,not implemented,33615,"5806 sw 112 wy tampa, fl"


### Test `miswrite_zipcode` function and vectorized version

That is, now that I figured out I need to filter down to 5-digit zipcodes first.

In [165]:
miswrite_zipcode_vec(zips, .2, .5, rng)

0    11224
1    89930
2    54783
dtype: object

In [266]:
miswrite_zipcode_vec(df_state_table.loc[five_digits, 'zipcode'], .2, .5, rng)

0        34591
1        04693
2        94665
3        44388
4        42403
         ...  
49995    33919
49996    30078
49997    32968
49998    33912
49999    32918
Name: zipcode, Length: 49979, dtype: object

In [193]:
miswrite_zipcode_vec(df_state_table.loc[five_digits, 'zipcode'], .2, .5, rng).isna().sum()

0

In [168]:
df_state_table.loc[five_digits, 'zipcode'].map(lambda z: miswrite_zipcode(z, .2, .5, rng))

0        34601
1        34698
2        34606
3        33628
4        02013
         ...  
49995    73911
49996    33004
49997    34985
49998    33989
49999    35168
Name: zipcode, Length: 49979, dtype: object

### Compare runtime of salar version of zipcode noise with `Series.map` vs. vectorized version using `Series.str` and NumPy

The vectorized version is about 4 times faster. Woo hoo!

In [267]:
%timeit miswrite_zipcode_vec(df_state_table.loc[five_digits, 'zipcode'], .2, .5, rng)

342 ms ± 27.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [171]:
%timeit df_state_table.loc[five_digits, 'zipcode'].map( \
    lambda z: miswrite_zipcode(z, .2, .5, rng))

1.34 s ± 113 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Check how long it takes to call `default_rng()`

As suspected, it takes longer to create a new Generator than to simply return an existing generator that is passed to the function. If the string functions in `corruption.py` or the ones above take a random state as a parameter, and they are passed an existing random Generator object, this would add about 266 ns per call according to the estimate below. If a single corruption function is called once on each of the 300 million records in the census, this adds about 80 seconds of total computation time.

To get reproducibility an alternative to optionally passing a random state would be to make the random Generator a *required* argument rather than optional. But for development rather than production, it's probably better to refactor the existing code with an optional random state so as not to break existing code, and to make it easier to write simple code for testing if it doesn't need to be reproducible.

In [73]:
%timeit np.random.default_rng()

18.7 µs ± 1.65 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [208]:
%timeit np.random.default_rng(sq)

9.63 µs ± 852 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [74]:
%timeit np.random.default_rng(rng)

266 ns ± 25.1 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [76]:
# 266 nanoseconds times 300 million records = 79.8 seconds
266e-9 * 300e6

79.80000000000001

In [77]:
300e6

300000000.0

## Write function to add noise to census data

In [180]:
# We'll need to filter out zipcodes > 5 digits to get the zipcode noise to work
df_census['zipcode'].str.len().value_counts()

5     47530
7        10
8         8
10        2
Name: zipcode, dtype: int64

In [248]:
# Filter out non-5-digit zipcodes
five_digit_zip = df_census['zipcode'].str.len() == 5
df_census_zip5 = df_census.loc[five_digit_zip]

In [261]:
def apply_noise_to_column(
    df, colname, row_prob, rng, noise_function, args=(), kwargs={}, inplace=False
):
    if not inplace:
        df = df.copy()
    """Apply a noise function that operates on scalars to a fraction of
    rows in a single column. Modifies column in place.
    """
    corrupted = rng.random(len(df)) < row_prob
    df.loc[corrupted, colname] = df.loc[corrupted, colname].map(
        lambda x: noise_function(x, *args, **kwargs))
    if not inplace:
        return df
        
def apply_vectorized_noise_to_column(
    df, colname, row_prob, rng, noise_function, args=(), kwargs={}, inplace=False
):
    if not inplace:
        df = df.copy()
    corrupted = rng.random(len(df)) < row_prob
    df.loc[corrupted, colname] = noise_function(df.loc[corrupted, colname], *args, **kwargs)
    if not inplace:
        return df

def add_noise_to_census(df_census, random_state):
    rng = np.random.default_rng(random_state)
    # Copy the dataframe since we're going to alter it
    df_census = df_census.copy()
    
    # First name
    apply_noise_to_column(
        df_census, 'first_name', 0.01, rng,
        corruption.keyboard_corrupt, (0.05, 0.02), inplace=True)
    apply_noise_to_column(
        df_census, 'first_name', 0.001, rng,
        corruption.phonetic_corrupt, (0.05,), inplace=True)
    
    # Last name
    apply_noise_to_column(
        df_census, 'last_name', 0.01, rng,
        corruption.keyboard_corrupt, (0.05, 0.02), inplace=True)
    apply_noise_to_column(
        df_census, 'last_name', 0.001, rng,
        corruption.phonetic_corrupt, (0.05,), inplace=True)
    
    # Date of birth
    apply_vectorized_noise_to_column(
        df_census, 'date_of_birth', 0.005, rng, swap_month_day, inplace=True)
    apply_noise_to_column(
        df_census, 'date_of_birth', 0.01, rng,
        corruption.keyboard_corrupt, (0.05, 0.02), inplace=True)
    
    # Zipcode
#     apply_noise_to_column(
#         df_census, 'zipcode', 0.01, rng,
#         miswrite_zipcode, (0.1, 0.2, rng), inplace=True)
    apply_vectorized_noise_to_column(
        df_census, 'zipcode', 0.01, rng,
        miswrite_zipcode_vec, (0.1, 0.2, rng), inplace=True)
#     apply_noise_to_column(
#         df_census, 'zipcode', .01, rng,
#         corruption.keyboard_corrupt, (0.05, 0.02), inplace=True)
    
    # Address
    apply_noise_to_column(
        df_census, 'address', 0.002, rng,
        corruption.keyboard_corrupt, (0.05, 0.02), inplace=True)
    apply_noise_to_column(
        df_census, 'address', 0.005, rng,
        corruption.phonetic_corrupt, (0.05,), inplace=True)
    
    # Sex
    ...
    
    # Age
    ...
    
    # Race/Ethnicity
    ...
    
    # Middle initial
    ...
    
    return df_census
    
df_census_noisy = add_noise_to_census(df_census_zip5, rng)
df_census_noisy

,first_name,middle_initial,last_name,date_of_birth,age,sex,race_ethnicity,relation_to_household_head,address,zipcode
0,Margaret,J,Clark,1951-07-27,68.0,Female,Black,Reference person,"1344 winoka rd brooksville, fl",34601
1,Jeffrey,V,Littlejohn,1967-05-03,52.0,Male,Black,Reference person,"927 23rd st clearwater, fl",34698
2,Briana,A,Jackson,2006-09-07,13.0,Female,Black,Biological child,"927 23rd st clearwater, fl",34698
3,Benjamin,D,Cox,1998-10-21,21.0,Male,Black,Stepchild,"927 23rd st clearwater, fl",34698
4,Willie,J,Tucker,1947-10-09,72.0,Male,White,Reference person,"8904 167th place fleming island, fl",32003
...,...,...,...,...,...,...,...,...,...,...
49995,Brian,N,Roberts,1984-07-05,35.0,Male,White,Institutionalized GQ pop,"114 s frnt st fort myers, fl",33919
49996,Nathaniel,J,Campbell,1941-01-08,79.0,Male,White,Institutionalized GQ pop,"2210 henn hyde rd ne hollywood, fl",33021
49997,Christian,C,Rosales,1983-12-16,36.0,Male,Latino,Institutionalized GQ pop,"701 haber rd vero beach, fl",32968
49998,Phillip,J,Morton,1985-06-11,34.0,Male,White,Institutionalized GQ pop,"114 s frnt st fort myers, fl",33919


In [263]:
df_census_zip5.compare(df_census_noisy)

first_name       last_name           date_of_birth              \
            self other      self     other          self       other   
37           NaN   NaN       NaN       NaN           NaN         NaN   
44           NaN   NaN       NaN       NaN    1991-04-23  1991-08-23   
153          NaN   NaN       NaN       NaN    1978-11-21  2978-11-21   
223          NaN   NaN       NaN       NaN           NaN         NaN   
295          NaN   NaN       NaN       NaN           NaN         NaN   
...          ...   ...       ...       ...           ...         ...   
49821        NaN   NaN       NaN       NaN           NaN         NaN   
49840        NaN   NaN       NaN       NaN    1999-07-31  1999-31-07   
49888        NaN   NaN  Shrestha  Shtestha           NaN         NaN   
49895        NaN   NaN       NaN       NaN    1966-06-17  1966-17-06   
49990        NaN   NaN       NaN       NaN           NaN         NaN   

                                   address  \
                                      self   
37                                     NaN   
44                                     NaN   
153                                    NaN   
223         3616 summerset pl  orlando, fl   
295                                    NaN   
...                                    ...   
49821  2210 henn hyde rd ne  hollywood, fl   
49840                                  NaN   
49888                                  NaN   
49895                                  NaN   
49990  2210 henn hyde rd ne  hollywood, fl   

                                           zipcode         
                                     other    self  other  
37                                     NaN   33440  34440  
44                                     NaN     NaN    NaN  
153                                    NaN     NaN    NaN  
223        3616 summerset pl  orlando, fle     NaN    NaN  
295                                    NaN   33056  35056  
...                                    ...     ...    ...  
49821   2210 henn yde rd ne  hllywood, fle     NaN    NaN  
49840                                  NaN     NaN    NaN  
49888                                  NaN     NaN    NaN  
49895                                  NaN     NaN    NaN  
49990  2210 hehn hyde rd ne  hollywood, fl     NaN    NaN  

[1122 rows x 10 columns]

In [246]:
df_census_noisy.loc[49740, 'zipcode']

'33919'

In [204]:
pd.Series('33919').str.split('')

0    [, 3, 3, 9, 1, 9, ]
dtype: object

In [264]:
df_census_noisy['zipcode'].isna().sum()

0

In [243]:
df_census.loc[five_digit_zip, 'zipcode'].compare(df_census_noisy['zipcode'])

,self,other
467,33541,33341
874,32128,32126
959,32955,36955
1657,32953,31953
1964,32765,32761
...,...,...
46681,33409,33405
47360,34953,234953
47780,33177,33157
47904,32233,36433


In [238]:
corruption.keyboard_corrupt('33919', 1, 1)

'2353692189'

In [239]:
corruption.keyboard_corrupt('a3dk7', 1, 1)

'sa23wdik87'

In [225]:
'3' in corruption.qwerty_error_dict

True

In [226]:
[d in corruption.qwerty_error_dict for d in '33919']

[True, True, True, True, True]

In [252]:
zip1 = miswrite_zipcode_vec(df_census_zip5['zipcode'], 0.1, 0.2, rng)
zip1

0        33601
1        34691
2        34698
3        34696
4        32003
         ...  
47525    33919
47526    33021
47527    32968
47528    33919
47529    32968
Length: 47530, dtype: object

In [254]:
zip1.isna().sum()

0

In [256]:
zip2 = apply_vectorized_noise_to_column(
        df_census_zip5, 'zipcode', 1, rng,
        miswrite_zipcode_vec, (0.1, 0.2, rng))
zip2

,first_name,middle_initial,last_name,date_of_birth,age,sex,race_ethnicity,relation_to_household_head,address,zipcode
0,Margaret,J,Clark,1951-07-27,68.0,Female,Black,Reference person,"1344 winoka rd brooksville, fl",34501
1,Jeffrey,V,Littlejohn,1967-05-03,52.0,Male,Black,Reference person,"927 23rd st clearwater, fl",37628
2,Briana,A,Jackson,2006-09-07,13.0,Female,Black,Biological child,"927 23rd st clearwater, fl",34698
3,Benjamin,D,Cox,1998-10-21,21.0,Male,Black,Stepchild,"927 23rd st clearwater, fl",34699
4,Willie,J,Tucker,1947-10-09,72.0,Male,White,Reference person,"8904 167th place fleming island, fl",34003
...,...,...,...,...,...,...,...,...,...,...
49995,Brian,N,Roberts,1984-07-05,35.0,Male,White,Institutionalized GQ pop,"114 s frnt st fort myers, fl",NaN
49996,Nathaniel,J,Campbell,1941-01-08,79.0,Male,White,Institutionalized GQ pop,"2210 henn hyde rd ne hollywood, fl",NaN
49997,Christian,C,Rosales,1983-12-16,36.0,Male,Latino,Institutionalized GQ pop,"701 haber rd vero beach, fl",NaN
49998,Phillip,J,Morton,1985-06-11,34.0,Male,White,Institutionalized GQ pop,"114 s frnt st fort myers, fl",NaN


In [258]:
zip2.loc[zip2['zipcode'].isna()]

,first_name,middle_initial,last_name,date_of_birth,age,sex,race_ethnicity,relation_to_household_head,address,zipcode
47530,Linda,C,Ornelas,1958-03-27,62.0,Female,Latino,Opp-sex spouse,"393 gaines st cpe coral, fl",NaN
47531,Hunter,Z,Lopez Munoz,1993-10-28,26.0,Male,Latino,Biological child,"393 gaines st cpe coral, fl",NaN
47532,Kathy,M,Gillespie,1953-08-21,66.0,Female,White,Reference person,"17 montgomery av lutz, fl",NaN
47533,Patricia,B,Chiesa,1956-07-19,63.0,Female,White,Reference person,"42 knighton drive apt 531 lutz, fl",NaN
47534,Robert,K,Posey,1950-09-16,69.0,Male,White,Opp-sex spouse,"42 knighton drive apt 531 lutz, fl",NaN
...,...,...,...,...,...,...,...,...,...,...
49995,Brian,N,Roberts,1984-07-05,35.0,Male,White,Institutionalized GQ pop,"114 s frnt st fort myers, fl",NaN
49996,Nathaniel,J,Campbell,1941-01-08,79.0,Male,White,Institutionalized GQ pop,"2210 henn hyde rd ne hollywood, fl",NaN
49997,Christian,C,Rosales,1983-12-16,36.0,Male,Latino,Institutionalized GQ pop,"701 haber rd vero beach, fl",NaN
49998,Phillip,J,Morton,1985-06-11,34.0,Male,White,Institutionalized GQ pop,"114 s frnt st fort myers, fl",NaN


# Testing, Testing, 1,2,3

## Test census generating function when state table contains dead simulants

In [58]:
temp_df = df_state_table.copy()
print((temp_df['cause_of_death'] == 'not_dead').sum())
grim_reaper = rng.random(len(temp_df)) < 0.3
temp_df.loc[grim_reaper, 'cause_of_death'] = 'scythe'
print((temp_df['cause_of_death'] == 'not_dead').sum())
test_df = datasets.generate_census_data(
    temp_df, overall_frac=0.8, kid_frac=0.6, random_state=rng)
test_df

50000
35105


,first_name,middle_initial,last_name,date_of_birth,age,sex,race_ethnicity,relation_to_household_head,address,zipcode
0,Margaret,J,Clark,1951-07-27,68.0,Female,Black,Reference person,"1344 winoka rd brooksville, fl",34601
1,Jeffrey,V,Littlejohn,1967-05-03,52.0,Male,Black,Reference person,"927 23rd st clearwater, fl",34698
2,Briana,A,Jackson,2006-09-07,13.0,Female,Black,Biological child,"927 23rd st clearwater, fl",34698
4,Willie,J,Tucker,1947-10-09,72.0,Male,White,Reference person,"8904 167th place fleming island, fl",32003
6,Judy,R,Long,1939-06-01,80.0,Female,White,Opp-sex spouse,"3478 claude douglas cir jacksonville, fl",32218
...,...,...,...,...,...,...,...,...,...,...
49992,Scott,J,Harris,1994-10-13,25.0,Male,White,Institutionalized GQ pop,"701 haber rd vero beach, fl",32968
49993,Thomas,B,Gutierrez,1990-11-16,29.0,Male,Latino,Institutionalized GQ pop,"2210 henn hyde rd ne hollywood, fl",33021
49996,Nathaniel,J,Campbell,1941-01-08,79.0,Male,White,Institutionalized GQ pop,"2210 henn hyde rd ne hollywood, fl",33021
49997,Christian,C,Rosales,1983-12-16,36.0,Male,Latino,Institutionalized GQ pop,"701 haber rd vero beach, fl",32968


In [59]:
35105/50000

0.7021

In [60]:
28025 / np.array([35105, 50000])

array([0.79831933, 0.5605    ])

# Initial attempt at more generalized noise function

I actually wrote this version before doing the more specific version above. Silly me.

In [ ]:
from vivarium_research_prl import noise

In [ ]:
noise_settings = {
    'corruption.keyboard_corrupt': [0.01, (0.05, 0.02)],
    'corruption.phonetic_corrupt': [0.001, (0.05,)],
}

def add_noise_to_column(column, noise_settings, random_state=None):
    if random_state is None:
        rng = np.random.default_rng(random_state)
    new_column = column.copy()
    for funckey, (row_prob, args) in noise_settings.items():
        module_name, funcname = funckey.split('.')
        func = getattr(getattr(noise, module_name), funcname)
        corrupted = rng.random(len(column)) < row_prob
        new_column.loc[corrupted] = new_column.loc[corrupted].map(lambda x: func(x, *args))
    return new_column

noised_names = add_noise_to_column(df_census.first_name, noise_settings)
noised_names

In [ ]:
df_census.first_name.compare(noised_names)

In [ ]:
dir(noise)